# Example on using the Beyond Quantum Noise Spectroscopy Framework

In [ ]:
# preample
from ipywidgets import widgets
from itertools import product
import pickle
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('./../src/')
from qubitmlmodel import  qubitMLmodel
from datasets_cat1 import sim_CPMG_G_SA

## Step 1: Create a dataset (skip if the dataset is already generated)

In [ ]:
# define the simulation parameters
T      = 1         # total time        
M      = 4096      # number of discrete time steps
Omega  = 10        # energy gap of the qubit

###########################################################################   
# generate single-axis dataset of Gaussian pulses 
print("Generating single-axis dataset of Gaussian Pulses")

# initialize the random number generator at some known point for reproducability
np.random.seed(seed = 40)

n_max       = 1  # maximum number of pulses 
parameters  = [(T, M, Omega, 1, 0,0, n_max)] + [(T, M, Omega, n_x, np.random.rand()*2, (2*np.random.rand(n_x)-1), n_max) for n_x,_ in product(range(1,n_max+1),range(75))]

training_results = []
for p in parameters:
    training_results.append(sim_CPMG_G_SA(*p))

parameters  =  [(T, M, Omega, n_x, np.random.rand()*2, (2*np.random.rand(n_x)-1), n_max) for n_x,_ in product(range(1,n_max+1),range(25))]
testing_results = []
for p in parameters:
    testing_results.append(sim_CPMG_G_SA(*p))

training_inputs  = [np.concatenate([x[0] for x in training_results], 0), np.concatenate([x[1] for x in training_results], 0)]
training_targets =  np.concatenate([x[2] for x in training_results], 0)    

testing_inputs   = [np.concatenate([x[0] for x in testing_results], 0), np.concatenate([x[1] for x in testing_results], 0)]
testing_targets  =  np.concatenate([x[2] for x in testing_results], 0)   

# store the dataset externally in a binary pickle file
f = open("./../datasets/CPMG_G_X_%d.ds"%n_max, 'wb')
pickle.dump({"T":T, "M":M, "Omega":Omega, "training_inputs":training_inputs, "training_targets":training_targets, "testing_inputs":testing_inputs, "testing_targets": testing_targets}, f, -1)
f.close()

## Step 2: load a dataset

In [ ]:
# 1) Load dataset
f = open("./../datasets/CPMG_G_X_28.ds", 'rb')
data = pickle.load(f)
f.close()  

# 2) Load all variables  
T                = data["T"]
M                = data["M"]
Omega            = data["Omega"]
training_inputs  = data["training_inputs"]
training_targets = data["training_targets"]
testing_inputs   = data["testing_inputs"]
testing_targets  = data["testing_targets"]

## Step 3: train a model (skip if the model is already trained)

In [ ]:
# 3)  Define the ML model
mlmodel = qubitMLmodel(T/M, Omega, "Single_Axis", 2)

# 4) Perform training 
mlmodel.train_model_val(training_inputs, training_targets, testing_inputs, testing_targets, 3000)    

# 5) Save results        
mlmodel.save_model("trained_model_CPMG_G_X_28_3000")

## Step 4: load a trained model

In [ ]:
# load the trained model
if testing_inputs[1].shape[-1] == 1:
    mlmodel = qubitMLmodel(T/M, Omega, "Single_Axis", testing_inputs[0].shape[-1])
else:
    mlmodel = qubitMLmodel(T/M, Omega, "Multi_Axis", testing_inputs[0].shape[-1]//2)    
mlmodel.load_model("trained_model_CPMG_G_X_28_3000")

## Step 5: display training history 

In [ ]:
plt.figure(figsize=[4.8, 3.8])
plt.loglog(mlmodel.training_history, label="Training")
plt.loglog(mlmodel.val_history, label="Validation")
plt.legend(fontsize=11)
plt.xlabel('Iteration', fontsize=11)
plt.ylabel('MSE',fontsize=11)
plt.xscale('log')
plt.xticks(sum([[i*j for i in range(1,11)] for j in [1,10,100,1000]],[]),fontsize=11)
plt.yticks(fontsize=11)
plt.grid(True, which="both")

# display the final MSE
print( "MSE for training set is %e"%mlmodel.model.evaluate(training_inputs,training_targets, batch_size = training_targets.shape[0])[0] )
print( "MSE for testing  set is %e"%mlmodel.model.evaluate(testing_inputs, testing_targets,  batch_size = testing_targets.shape[0])[0] )

## Step 6: display some testing examples

In [ ]:
time_range = np.array([(0.5*T/M) + (j*T/M) for j in range(M)]) # time_domain vector

# use the trained model to predict measurements of the training set
predicted_measurements_testing  = mlmodel.predict_measurements(testing_inputs)

# define a function to display a particular example      
def update_display(idx_example):
    plt1 = testing_inputs[1][idx_example,:]
    plt2 = testing_targets[idx_example,:]
    plt3 = predicted_measurements_testing[idx_example,:]

    plt.figure(figsize=[8, 6])
    plt.subplot(2,1,1)
    plt.plot(time_range, plt1[:,0],'r',label='h_x(t)')
    plt.xlabel('t',fontsize=11)
    plt.ylabel("$\mathbf{h}(t)$",fontsize=11)
    plt.grid()
    plt.legend(fontsize=11)
        
    ax = plt.subplot(2,1,2)
    plt.plot(plt2, ".", label = "actual")
    plt.plot(plt3, "x", label = "predicted")
    plt.xlabel('Initial State/Measurement Operator',fontsize=11)
    plt.xticks(fontsize=11)
    labels = [ [r"$\rho_{%s}, \sigma_{%s}$"%(rho,O) for O in ["x","y","z"]] for rho in ["x+","x-","y+","y-","z+","z-"]]
    labels = sum(labels,[])
    ax.set_xticks([x for x in range(18)])
    ax.set_xticklabels(labels, rotation=90)
    plt.ylim([-1.1, 1.1])
    plt.ylabel("Expectation",fontsize=11)
    plt.yticks(fontsize=11)
    plt.grid()
    plt.legend(fontsize=11)   
    plt.tight_layout()

In [ ]:
# add a widget for selecting the example
widgets.interact(update_display, idx_example=widgets.IntSlider(min=0,max=testing_targets.shape[0]-1,step=1, continuous_update=False) )
# or display an example directly 
#update_display(0)
#update_display(381)
#update_display(470)

## Step 7: perform quantum control

In [ ]:
n_max = 28

G_names= ["I", "X", "Y", "Z"]     
G = [np.array([[1.,0.],[0.,1.]]), np.array([[0.,1.],[1.,0.]]), np.array([[0.,-1j],[1j,0.]]), np.array([[1.,0],[0.,-1.]])]

for idx_G, G in enumerate(G):
    mlmodel.construct_controller(T, M, n_max)
    pulses = mlmodel.train_controller(G,1000)

    # plot the results
    plt.figure(figsize=[8, 6])
    plt.subplot(2,1,1)
    plt.plot(time_range, pulses[1][0,:,0],'r',label='h_x(t)')
    plt.xlabel('t',fontsize=11)
    plt.ylabel("$\mathbf{h}(t)$",fontsize=11)
    plt.grid()
    plt.legend(fontsize=11)
    print("Fidelities for gate %s are %f %f, %f, %f\n"%(G_names[idx_G], 100*(1-mlmodel.controller_training_history["Fid_0_loss"][-1]), 100*(1-mlmodel.controller_training_history["Fid_1_loss"][-1]), 100*(1-mlmodel.controller_training_history["Fid_2_loss"][-1]), 100*(1-mlmodel.controller_training_history["Fid_3_loss"][-1])))